<a href="https://colab.research.google.com/github/lightrao/google_colab_files/blob/master/ai-agent-crewai/L2_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [4]:
%pip uninstall numpy
%pip install numpy==1.25.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 58.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [6]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [18]:
import os

# from utils import get_openai_api_key
# openai_api_key = get_openai_api_key()

from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [19]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True
)

### Agent: Writer

In [20]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [21]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [22]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [23]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [24]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [25]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [26]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Unveiling the Latest Trends in Artificial Intelligence: A Comprehensive Guide

Outline:

I. Introduction
- Brief overview of Artificial Intelligence (AI)
- Importance of staying updated on AI trends
- Thesis statement: Exploring the latest trends, key players, and noteworthy news in the world of AI

II. Latest Trends in Artificial Intelligence
A. Machine Learning advancements
B. Natural Language Processing (NLP) breakthroughs
C. Robotics and Automation innovations
D. Ethical considerat

- Display the results of your execution as markdown in the notebook.

In [27]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction
Artificial Intelligence (AI) has rapidly become a key player in various industries, revolutionizing the way we interact with technology. Staying abreast of the latest trends in AI is crucial for tech enthusiasts, business owners, students, and researchers alike. In this comprehensive guide, we delve into the recent advancements, key players, noteworthy news, and ethical considerations in the world of AI.

## Latest Trends in Artificial Intelligence
### Machine Learning Advancements
Machine Learning continues to be a driving force behind AI innovation, with breakthroughs in deep learning algorithms, neural networks, and predictive analytics. These advancements have enabled AI systems to learn from data, identify patterns, and make decisions without explicit programming.

### Natural Language Processing (NLP) Breakthroughs
NLP has seen significant progress in recent years, with the development of language models like GPT-3 and BERT. These models have improved language understanding, translation, and text generation capabilities, paving the way for more sophisticated AI-powered applications.

### Robotics and Automation Innovations
The integration of AI in robotics and automation has led to the creation of autonomous systems capable of performing complex tasks with precision and efficiency. From self-driving cars to robotic surgery, AI has transformed the way we interact with machines.

### Ethical Considerations in AI Development
As AI technologies become more pervasive, ethical considerations around data privacy, bias, and accountability have come to the forefront. It is essential for developers and policymakers to address these ethical challenges to ensure the responsible development and deployment of AI systems.

## Key Players in the AI Industry
### Google's DeepMind
DeepMind, acquired by Google in 2014, is renowned for its advancements in AI research, particularly in the fields of deep reinforcement learning and neural network algorithms. Their work has contributed significantly to the development of AI applications in healthcare, gaming, and robotics.

### IBM Watson
IBM Watson is a pioneer in cognitive computing, leveraging AI and machine learning to analyze large datasets, extract insights, and facilitate decision-making. Watson's capabilities span various industries, including healthcare, finance, and customer service.

### Microsoft Azure AI
Microsoft's Azure AI platform offers a suite of AI services and tools for developers to build, deploy, and manage AI solutions. From computer vision to natural language understanding, Azure AI enables businesses to harness the power of AI for improved efficiency and innovation.

### OpenAI
OpenAI is a research organization dedicated to advancing AI in a safe and beneficial manner. Known for developing cutting-edge AI models like GPT-3, OpenAI focuses on ethical AI development, transparency, and collaboration with the broader AI community.

## Noteworthy News in Artificial Intelligence
### Recent AI Applications in Healthcare
AI-powered healthcare solutions have transformed patient care, diagnosis, and treatment, with applications ranging from medical imaging analysis to personalized medicine. These advancements have the potential to improve healthcare outcomes and reduce costs.

### AI-Powered Solutions for Businesses
Businesses are increasingly turning to AI for process automation, predictive analytics, and customer insights. AI technologies like chatbots, recommendation systems, and predictive maintenance are helping organizations streamline operations and drive innovation.

### Regulatory Updates in the AI Landscape
Regulatory bodies are grappling with the complex challenges posed by AI, including data privacy, algorithmic bias, and AI transparency. Recent updates in AI regulations aim to promote responsible AI development and mitigate potential risks associated with AI deployment.

### Impact of AI on Job Market
The integration of AI technologies in the workforce has led to concerns about job displacement and skills retraining. While AI has the potential to create new job opportunities and enhance productivity, it also raises questions about the future of work and the need for upskilling in AI-related fields.

## Target Audience Analysis
Our guide caters to a diverse audience interested in AI, including tech enthusiasts, business owners, students, and researchers. Whether you are looking to implement AI solutions in your business, stay informed about the latest trends, or explore AI courses for further learning, this guide provides valuable insights and resources to support your AI journey.

## Call to Action
Stay informed by subscribing to AI newsletters, engage in discussions on AI forums and communities, and explore AI courses and certifications for continuous learning and professional development. Join the conversation and be part of the exciting world of Artificial Intelligence!

By following this comprehensive guide, we aim to empower our readers with the knowledge and resources needed to navigate the dynamic landscape of Artificial Intelligence. Let's embark on this AI journey together and unlock the endless possibilities that AI has to offer.

*SEO Keywords: Artificial Intelligence trends 2021, Top AI companies, Latest advancements in Machine Learning, Ethical AI development, AI news updates*

*Resources:
- Forbes: [https://www.forbes.com/artificial-intelligence-trends](https://www.forbes.com/artificial-intelligence-trends)
- TechCrunch: [https://techcrunch.com/top-ai-companies](https://techcrunch.com/top-ai-companies)
- MIT Technology Review: [https://www.technologyreview.com/ai-news-updates](https://www.technologyreview.com/ai-news-updates)*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).